# Natural Language SQL processor 
### with HuggingFace's free, open source Inference API

---

Get a free Hugging Face Inference API Access token with [these instructions](https://huggingface.co/docs/hub/security-tokens).

I use Hugging Face's [Zephyr 7B Beta](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta) large language model. There are probably better ones, but this one is small? I think?

The default database is MySQL. To change, configure the uri in #2.

### Set configuration

In [128]:
DB_NAME = "..."
USER = "..."
PWD = "..."
HOST = "..."

HUGGING_FACE_ACCESS_TOKEN = "..."

QUESTION = "How many orders were placed on september 10th, 2023?"

## 1. Install

In [129]:
install = False # change to True if you want to install the dependencies
if install:
    !pip install sqlalchemy
    !pip install langchain
    !pip install mysql-connector-python
    !pip install huggingface_hub
    !pip install langchain-experimental

## 2. Connect to MySQL DB

In [130]:
import sqlalchemy
from langchain.utilities import SQLDatabase

db_uri = f"mysql+mysqlconnector://{USER}:{PWD}@{HOST}/{DB_NAME}"  # change this for other dbs

In [131]:
db = SQLDatabase.from_uri(db_uri)

## 3. Connect to Hugging Face Chat model

In [132]:
import os
from langchain_community.llms import HuggingFaceHub

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGING_FACE_ACCESS_TOKEN

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta"
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


## 4. Get SQL response

`create_sql_query_chain` does not create an agent (I don't think), but just a runnable 'chain' object that fills and runs [pre-engineered prompts](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/sql_database/prompt.py) against the LLM.

In [133]:
from langchain.chains import create_sql_query_chain

sql_chain = create_sql_query_chain(llm, db)

In [135]:
# this takes like a minute

sql_response = sql_chain.invoke({"question": QUESTION})

In [136]:
sql_response

"SELECT COUNT(*) FROM orders WHERE `createdTime` LIKE '2023-09-10%';"

## 5. Execute query

In [137]:
# create sqlalchemy connection for real

engine = sqlalchemy.create_engine(db_uri)
connection = engine.connect()

In [138]:
import pandas as pd

# execute the query, return as dataframe

df = pd.read_sql_query(
    sql = sql_response,
    con = engine
)

In [139]:
df

,COUNT(*)
0,1816
